# Folder tree size
Navigates a folder tree up to a given depth, and prints corresponding sizes.

In [46]:
import subprocess
import re
import os

In [125]:
# settings
folder = "/scratch/gwf/gwf_cmt/wknoben/camels-spat-upload"
depth = 4

In [126]:
import subprocess
import re
import os

### --- RUN SUBPROCESSES --- ###
def run_tree(directory=".", depth=2):
    """Runs the 'tree' command and returns raw output."""
    result = subprocess.run(["tree", "-d", "-L", str(depth), directory], capture_output=True, text=True)
    return result.stdout  # Return raw output

def run_du(directory=".", depth=2):
    """Runs the 'du' command and returns raw output."""
    result = subprocess.run(["du", "-h", "--max-depth", str(depth), directory], capture_output=True, text=True)
    return result.stdout  # Return raw output

### --- CLEANUP & PROCESSING FUNCTIONS --- ###
def remove_ansi_escape(text):
    """Removes ANSI color codes from text."""
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    return ansi_escape.sub('', text)

def process_tree_output(raw_tree_output, directory):
    """Processes raw tree output into (full_path, formatted_line) using indentation."""
    tree_lines = remove_ansi_escape(raw_tree_output).split("\n")
    
    tree_structure = []
    base_path = os.path.abspath(directory)  # Convert base path to absolute
    path_stack = [base_path]  # Track hierarchy using a stack
    last_level = 0
    for line in tree_lines:
        
        # Ignore empty lines or summary lines
        if not line.strip() or "directories" in line:
            continue
    
        # Count indentation depth (│ or spaces)
        depth_level = line.count("│") + line.count("├") + line.count("└") + last_level
    
        # Extract folder name and reconstruct the full path
        folder_name = line.strip().split("─")[-1].strip()
    
        # Start the whole thing of
        if depth_level == 0:
            full_path = os.path.join(path_stack[-1], folder_name)
            full_path = os.path.normpath(full_path)  # Normalize to remove redundant slashes
        else: # depth > 0
            if depth_level < len(path_stack):
                path_stack = path_stack[:depth_level + 1]  # Truncate the stack to correct level
    
        # Make the path
        full_path = os.path.join(path_stack[-1], folder_name)
        full_path = os.path.normpath(full_path)  # Normalize to remove redundant slashes
        path_stack.append(full_path)  # Store new path at the correct level
    
        # housekeeping
        last_depth = depth_level
        if folder_name == 'shapefiles':
            last_level = 1 # final entries has fever indentation characters, so we add one
    
        # track results
        tree_structure.append((full_path, line))  # Store (full_path, tree_line)

    return tree_structure

def process_du_output(raw_du_output):
    """Processes raw 'du' output into a dictionary mapping full paths to sizes."""
    size_dict = {}
    for line in raw_du_output.split("\n"):
        parts = line.split("\t")
        if len(parts) == 2:
            size, path = parts
            clean_path = os.path.normpath(path.strip())  # Normalize path for comparison
            size_dict[clean_path] = size  # Store full path as key

    return size_dict

def align_tree_with_sizes(tree_structure, du_sizes):
    """Aligns tree structure with disk usage sizes and formats output."""
    formatted_output = []

    # Find max tree line length for alignment
    max_tree_length = max(len(re.sub(r'[\s─│]+', '', line.strip())) for _, line in tree_structure) + 5
    size_column_start = max_tree_length + 5  # Adjust spacing between tree & size

    for full_path, tree_line in tree_structure:
        # Ensure exact match of full path
        folder_size = du_sizes.get(full_path, "N/A")
        
        # Align size brackets to a fixed column
        formatted_output.append(f"{tree_line}".ljust(size_column_start) + f"[{folder_size}]")

    return "\n".join(formatted_output)

In [127]:
# Get both outputs
tree_result = subprocess.run(["tree", "-d", "-L", f"{depth}", folder], capture_output=True, text=True)
du_result = subprocess.run(["du", "-h", f"--max-depth={depth}", folder], capture_output=True, text=True)

In [130]:
# Process outputs
tree_lines = remove_ansi_escape(tree_result.stdout).split("\n")
du_sizes = process_du_output(du_result.stdout)

```
CAMELS-SPAT                                              [5.5T]
├── attributes                                           [650M]
│   ├── headwater                                        [20M]
│   ├── macro-scale                                      [453M]
│   └── meso-scale                                       [156M]
├── forcing                                              [5.4T]
│   ├── headwater                                        [417G]
│   │   ├── daymet                                       [27G]
│   │   │   ├── daymet-distributed                       [203M]
│   │   │   ├── daymet-gridded                           [27G]
│   │   │   └── daymet-lumped                            [202M]
│   │   ├── em-earth                                     [7.9G]
│   │   │   ├── em-earth-distributed                     [2.2G]
│   │   │   ├── em-earth-gridded                         [3.7G]
│   │   │   └── em-earth-lumped                          [2.1G]
│   │   ├── era5                                         [33G]
│   │   │   ├── era5-distributed                         [8.8]
│   │   │   ├── era5-gridded                             [15G]
│   │   │   ├── era5-invariants                          [19M]
│   │   │   └── era5-lumped                              [8.8G]
│   │   └── rdrs                                         [351G]
│   │   │   ├── rdrs-distributed                         [5.5G]
│   │   │   ├── rdrs-gridded                             [340G]
│   │   │   └── rdrs-lumped                              [5.5G]
│   ├── macro-scale                                      [3.2T]
│   │   ├── daymet                                       [776G]
│   │   │   ├── daymet-distributed                       [7.3G]
│   │   │   ├── daymet-gridded                           [769G]
│   │   │   └── daymet-lumped                            [273M]
│   │   ├── em-earth                                     [127G]
│   │   │   ├── em-earth-distributed                     [53G]  
│   │   │   ├── em-earth-gridded                         [72G]
│   │   │   └── em-earth-lumped                          [3.2G]
│   │   ├── era5                                         [671G]
│   │   │   ├── era5-distributed                         [490G]
│   │   │   ├── era5-gridded                             [169G]
│   │   │   ├── era5-invariants                          [25M]
│   │   │   └── era5-lumped                              [13G]
│   │   └── rdrs                                   [1.7T]
│   │   │   ├── rdrs-distributed                         [293G]
│   │   │   ├── rdrs-gridded                             [1.4T]
│   │   │   └── rdrs-lumped                              [7.6G]
│   └── meso-scale                                 [1.8T]
│       ├── daymet                                       [252G]
│       │   ├── daymet-distributed                       [2.1G]
│       │   ├── daymet-gridded                           [249G]
│       │   └── daymet-lumped                            [496M]
│       ├── em-earth                                     [46G]
│       │   ├── em-earth-distributed                     [17G]
│       │   ├── em-earth-gridded                         [25G]  
│       │   └── em-earth-lumped                          [5.7G]
│       ├── era5                                         [233G]
│       │   ├── era5-distributed                         [127G]
│       │   ├── era5-gridded                             [83G]
│       │   ├── era5-invariants                          [45M]
│       │   └── era5-lumped                              [24G]
│       └── rdrs                                         [1.3T]
│           ├── rdrs-distributed                         [76G]
│           ├── rdrs-gridded                             [1.2T]
│           └── rdrs-lumped                              [14G]
├── geospatial                                           [119G]
│   ├── headwater                                        [2.0G]
│   │   ├── forest-height                                [152M]
│   │   ├── glclu2019                                    [153M]
│   │   ├── glhymps                                      [9.9M]
│   │   ├── hydrolakes                                   [2.2M]
│   │   ├── lai                                          [396M]
│   │   ├── lgrip30                                      [67M]
│   │   ├── merit                                        [124M]
│   │   ├── modis-land                                   [11M]
│   │   ├── pelletier                                    [1.5M]
│   │   ├── soilgrids                                    [1.1G]
│   │   └── worldclim-derived                            [11M]
│   ├── macro-scale                                      [86G]
│   │   ├── forest-height                                [11G]
│   │   ├── glclu2019                                    [11G]
│   │   ├── glhymps                                      [109M]
│   │   ├── hydrolakes                                   [76M]
│   │   ├── lai                                          [28G]
│   │   ├── lgrip30                                      [4.5G]
│   │   ├── merit                                        [7.9G]
│   │   ├── modis-land                                   [716M]
│   │   ├── pelletier                                    [100M]
│   │   ├── soilgrids                                    [25G]
│   │   └── worldclim-derived                            [359M]
│   ├── meso-scale                                       [31G]
│   │   ├── forest-height                                [2.2G]
│   │   ├── glclu2019                                    [2.2G]
│   │   ├── glhymps                                      [54M]
│   │   ├── hydrolakes                                   [14M]
│   │   ├── lai                                          [17G]
│   │   ├── lgrip30                                      [954M]
│   │   ├── merit                                        [1.7G]
│   │   ├── modis-land                                   [380M]
│   │   ├── pelletier                                    [36M]
│   │   ├── soilgrids                                    [7.2G]
│   │   └── worldclim-derived                            [122M]
│   └── _metadata                                        [512]
├── observations                                         [8.0G]
│   ├── headwater                                        [1.9G]
│   │   ├── obs-daily                                    [191M]
│   │   └── obs-hourly                                   [1.7G]
│   ├── macro-scale                                      [1.7G]
│   │   ├── obs-daily                                    [280M]
│   │   └── obs-hourly                                   [1.5G]
│   └── meso-scale                                       [4.5G]
│       ├── obs-daily                                    [N/A]
│       └── obs-hourly                                   [N/A]
└── shapefiles                                           [2.8G]
    ├── headwater                                        [209M]
    │   ├── delineation-outcomes                         [113M]
    │   ├── shapes-distributed                           [11M]
    │   ├── shapes-forcing                               [65M]
    │   ├── shapes-lumped                                [8.2M]
    │   └── shapes-reference                             [13M]
    ├── macro-scale                                      [1.7G]
    │   ├── delineation-outcomes                         [235M]
    │   ├── shapes-distributed                           [264M]
    │   ├── shapes-forcing                               [1.1G]
    │   ├── shapes-lumped                                [34M]
    │   └── shapes-reference                             [77M]
    └── meso-scale                                       [898M]
        ├── delineation-outcomes                         [320M]
        ├── shapes-distributed                           [87M]
        ├── shapes-forcing                               [404M]
        ├── shapes-lumped                                [27M]
        └── shapes-reference                             [61M]
```

```
CAMELS-SPAT                               [5.5T]
├── attributes                             ├── [650M]
│   ├── headwater                          │   ├── [20M]
│   ├── macro-scale                        │   ├── [453M]
│   └── meso-scale                         │   └── [156M]
├── forcing                                ├── [5.4T]
│   ├── headwater                          │   ├── [417G]
│   │   ├── daymet                         │   │   ├── [27G]
│   │   │   ├── daymet-distributed         │   │   │   ├── [203M]
│   │   │   ├── daymet-gridded             │   │   │   ├── [27G]
│   │   │   └── daymet-lumped              │   │   │   └── [202M]
│   │   ├── em-earth                       │   │   ├── [7.9G]
│   │   │   ├── em-earth-distributed       │   │   │   ├── [2.2G]
│   │   │   ├── em-earth-gridded           │   │   │   ├── [3.7G]
│   │   │   └── em-earth-lumped            │   │   │   └── [2.1G]
│   │   ├── era5                           │   │   ├── [33G]
│   │   │   ├── era5-distributed           │   │   │   ├── [8.8]
│   │   │   ├── era5-gridded               │   │   │   ├── [15G]
│   │   │   ├── era5-invariants            │   │   │   ├── [19M]
│   │   │   └── era5-lumped                │   │   │   └── [8.8G]
│   │   └── rdrs                           │   │   └── [351G]
│   │       ├── rdrs-distributed           │   │   │   ├── [5.5G]
│   │       ├── rdrs-gridded               │   │   │   ├── [340G]
│   │       └── rdrs-lumped                │   │   │   └── [5.5G]
│   ├── macro-scale                        │   ├── [3.2T]
│   │   ├── daymet                         │   │   ├── [776G]
│   │   │   ├── daymet-distributed         │   │   │   ├── [7.3G]
│   │   │   ├── daymet-gridded             │   │   │   ├── [769G]
│   │   │   └── daymet-lumped              │   │   │   └── [273M]
│   │   ├── em-earth                       │   │   ├── [127G]
│   │   │   ├── em-earth-distributed       │   │   │   ├── [53G]  
│   │   │   ├── em-earth-gridded           │   │   │   ├── [72G]
│   │   │   └── em-earth-lumped            │   │   │   └── [3.2G]
│   │   ├── era5                           │   │   ├── [671G]
│   │   │   ├── era5-distributed           │   │   │   ├── [490G]
│   │   │   ├── era5-gridded               │   │   │   ├── [169G]
│   │   │   ├── era5-invariants            │   │   │   ├── [25M]
│   │   │   └── era5-lumped                │   │   │   └── [13G]
│   │   └── rdrs                           │   │   └── [1.7T]
│   │       ├── rdrs-distributed           │   │       ├── [293G]
│   │       ├── rdrs-gridded               │   │       ├── [1.4T]
│   │       └── rdrs-lumped                │   │       └── [7.6G]
│   └── meso-scale                         │   └── [1.8T]
│       ├── daymet                         │       ├── [252G]
│       │   ├── daymet-distributed         │       │   ├── [2.1G]
│       │   ├── daymet-gridded             │       │   ├── [249G]
│       │   └── daymet-lumped              │       │   └── [496M]
│       ├── em-earth                       │       ├── [46G]
│       │   ├── em-earth-distributed       │       │   ├── [17G]
│       │   ├── em-earth-gridded           │       │   ├── [25G]  
│       │   └── em-earth-lumped            │       │   └── [5.7G]
│       ├── era5                           │       ├── [233G]
│       │   ├── era5-distributed           │       │   ├── [127G]
│       │   ├── era5-gridded               │       │   ├── [83G]
│       │   ├── era5-invariants            │       │   ├── [45M]
│       │   └── era5-lumped                │       │   └── [24G]
│       └── rdrs                           │       └── [1.3T]
│           ├── rdrs-distributed           │           ├── [76G]
│           ├── rdrs-gridded               │           ├── [1.2T]
│           └── rdrs-lumped                │           └── [14G]
├── geospatial                             ├── [119G]
│   ├── headwater                          │   ├── [2.0G]
│   │   ├── forest-height                  │   │   ├── [152M]
│   │   ├── glclu2019                      │   │   ├── [153M]
│   │   ├── glhymps                        │   │   ├── [9.9M]
│   │   ├── hydrolakes                     │   │   ├── [2.2M]
│   │   ├── lai                            │   │   ├── [396M]
│   │   ├── lgrip30                        │   │   ├── [67M]
│   │   ├── merit                          │   │   ├── [124M]
│   │   ├── modis-land                     │   │   ├── [11M]
│   │   ├── pelletier                      │   │   ├── [1.5M]
│   │   ├── soilgrids                      │   │   ├── [1.1G]
│   │   └── worldclim-derived              │   │   └── [11M]
│   ├── macro-scale                        │   ├── [86G]
│   │   ├── forest-height                  │   │   ├── [11G]
│   │   ├── glclu2019                      │   │   ├── [11G]
│   │   ├── glhymps                        │   │   ├── [109M]
│   │   ├── hydrolakes                     │   │   ├── [76M]
│   │   ├── lai                            │   │   ├── [28G]
│   │   ├── lgrip30                        │   │   ├── [4.5G]
│   │   ├── merit                          │   │   ├── [7.9G]
│   │   ├── modis-land                     │   │   ├── [716M]
│   │   ├── pelletier                      │   │   ├── [100M]
│   │   ├── soilgrids                      │   │   ├── [25G]
│   │   └── worldclim-derived              │   │   └── [359M]
│   ├── meso-scale                         │   ├── [31G]
│   │   ├── forest-height                  │   │   ├── [2.2G]
│   │   ├── glclu2019                      │   │   ├── [2.2G]
│   │   ├── glhymps                        │   │   ├── [54M]
│   │   ├── hydrolakes                     │   │   ├── [14M]
│   │   ├── lai                            │   │   ├── [17G]
│   │   ├── lgrip30                        │   │   ├── [954M]
│   │   ├── merit                          │   │   ├── [1.7G]
│   │   ├── modis-land                     │   │   ├── [380M]
│   │   ├── pelletier                      │   │   ├── [36M]
│   │   ├── soilgrids                      │   │   ├── [7.2G]
│   │   └── worldclim-derived              │   │   ├── [122M]
│   └── _metadata                          │   └── [3.5M]
├── observations                           ├── [8.0G]
│   ├── headwater                          │   ├── [1.9G]
│   │   ├── obs-daily                      │   │   ├── [191M]
│   │   └── obs-hourly                     │   │   └── [1.7G]
│   ├── macro-scale                        │   ├── [1.7G]
│   │   ├── obs-daily                      │   │   ├── [280M]
│   │   └── obs-hourly                     │   │   └── [1.5G]
│   └── meso-scale                         │   ├── [4.5G]
│       ├── obs-daily                      │   │   ├── [502M]
│       └── obs-hourly                     │   │   └── [4G]
└── shapefiles                             └── [2.8G]
    ├── headwater                              ├── [209M]
    │   ├── delineation-outcomes               │   ├── [113M]
    │   ├── shapes-distributed                 │   ├── [11M]
    │   ├── shapes-forcing                     │   ├── [65M]
    │   ├── shapes-lumped                      │   ├── [8.2M]
    │   └── shapes-reference                   │   └── [13M]
    ├── macro-scale                            ├── [1.7G]
    │   ├── delineation-outcomes               │   ├── [235M]
    │   ├── shapes-distributed                 │   ├── [264M]
    │   ├── shapes-forcing                     │   ├── [1.1G]
    │   ├── shapes-lumped                      │   ├── [34M]
    │   └── shapes-reference                   │   └── [77M]
    └── meso-scale                             └── [898M]
        ├── delineation-outcomes                    ├── [320M]
        ├── shapes-distributed                      ├── [87M]
        ├── shapes-forcing                          ├── [404M]
        ├── shapes-lumped                           ├── [27M]
        └── shapes-reference                        └── [61M]
```

```
CAMELS-SPAT                               [5500G]
├── attributes                             ├── [0.65G]
│   ├── headwater                          │   ├── [0.002G]
│   ├── macro-scale                        │   ├── [0.453G]
│   └── meso-scale                         │   └── [0.156G]
├── forcing                                ├── [5400G]
│   ├── headwater                          │   ├── [417G]
│   │   ├── daymet                         │   │   ├── [27G]
│   │   │   ├── daymet-distributed         │   │   │   ├── [ 0.203G]
│   │   │   ├── daymet-gridded             │   │   │   ├── [27G]
│   │   │   └── daymet-lumped              │   │   │   └── [ 0.202G]
│   │   ├── em-earth                       │   │   ├── [7.9G]
│   │   │   ├── em-earth-distributed       │   │   │   ├── [2.2G]
│   │   │   ├── em-earth-gridded           │   │   │   ├── [3.7G]
│   │   │   └── em-earth-lumped            │   │   │   └── [2.1G]
│   │   ├── era5                           │   │   ├── [33G]
│   │   │   ├── era5-distributed           │   │   │   ├── [ 8.8G]
│   │   │   ├── era5-gridded               │   │   │   ├── [15G]
│   │   │   ├── era5-invariants            │   │   │   ├── [ 0.0019G]
│   │   │   └── era5-lumped                │   │   │   └── [ 8.8G]
│   │   └── rdrs                           │   │   └── [351G]
│   │       ├── rdrs-distributed           │   │   │   ├── [  5.5G]
│   │       ├── rdrs-gridded               │   │   │   ├── [340G]
│   │       └── rdrs-lumped                │   │   │   └── [  5.5G]
│   ├── macro-scale                        │   ├── [3200G]
│   │   ├── daymet                         │   │   ├── [776G]
│   │   │   ├── daymet-distributed         │   │   │   ├── [  7.3G]
│   │   │   ├── daymet-gridded             │   │   │   ├── [769G]
│   │   │   └── daymet-lumped              │   │   │   └── [  0.273G]
│   │   ├── em-earth                       │   │   ├── [127G]
│   │   │   ├── em-earth-distributed       │   │   │   ├── [53G]  
│   │   │   ├── em-earth-gridded           │   │   │   ├── [72G]
│   │   │   └── em-earth-lumped            │   │   │   └── [ 3.2G]
│   │   ├── era5                           │   │   ├── [671G]
│   │   │   ├── era5-distributed           │   │   │   ├── [490G]
│   │   │   ├── era5-gridded               │   │   │   ├── [169G]
│   │   │   ├── era5-invariants            │   │   │   ├── [  0.025G]
│   │   │   └── era5-lumped                │   │   │   └── [ 13G]
│   │   └── rdrs                           │   │   └── [1700G]
│   │       ├── rdrs-distributed           │   │       ├── [ 293G]
│   │       ├── rdrs-gridded               │   │       ├── [1400G]
│   │       └── rdrs-lumped                │   │       └── [   7.6G]
│   └── meso-scale                         │   └── [1800G]
│       ├── daymet                         │       ├── [252G]
│       │   ├── daymet-distributed         │       │   ├── [  2.1G]
│       │   ├── daymet-gridded             │       │   ├── [249G]
│       │   └── daymet-lumped              │       │   └── [  0.496G]
│       ├── em-earth                       │       ├── [46G]
│       │   ├── em-earth-distributed       │       │   ├── [17G]
│       │   ├── em-earth-gridded           │       │   ├── [25G]  
│       │   └── em-earth-lumped            │       │   └── [ 5.7G]
│       ├── era5                           │       ├── [233G]
│       │   ├── era5-distributed           │       │   ├── [127G]
│       │   ├── era5-gridded               │       │   ├── [ 83G]
│       │   ├── era5-invariants            │       │   ├── [  0.045G]
│       │   └── era5-lumped                │       │   └── [ 24G]
│       └── rdrs                           │       └── [1300T]
│           ├── rdrs-distributed           │           ├── [  76G]
│           ├── rdrs-gridded               │           ├── [1200G]
│           └── rdrs-lumped                │           └── [  14G]
├── geospatial                             ├── [119G]
│   ├── headwater                          │   ├── [2.0G]
│   │   ├── forest-height                  │   │   ├── [0.152G]
│   │   ├── glclu2019                      │   │   ├── [0.153G]
│   │   ├── glhymps                        │   │   ├── [0.0099G]
│   │   ├── hydrolakes                     │   │   ├── [0.0022G]
│   │   ├── lai                            │   │   ├── [0.396G]
│   │   ├── lgrip30                        │   │   ├── [0.067G]
│   │   ├── merit                          │   │   ├── [0.124G]
│   │   ├── modis-land                     │   │   ├── [0.011G]
│   │   ├── pelletier                      │   │   ├── [0.0015G]
│   │   ├── soilgrids                      │   │   ├── [1.1G]
│   │   └── worldclim-derived              │   │   └── [0.011G]
│   ├── macro-scale                        │   ├── [86G]
│   │   ├── forest-height                  │   │   ├── [11G]
│   │   ├── glclu2019                      │   │   ├── [11G]
│   │   ├── glhymps                        │   │   ├── [ 0.109G]
│   │   ├── hydrolakes                     │   │   ├── [ 0.076G]
│   │   ├── lai                            │   │   ├── [28G]
│   │   ├── lgrip30                        │   │   ├── [ 4.5G]
│   │   ├── merit                          │   │   ├── [ 7.9G]
│   │   ├── modis-land                     │   │   ├── [ 0.716G]
│   │   ├── pelletier                      │   │   ├── [ 0.100G]
│   │   ├── soilgrids                      │   │   ├── [25G]
│   │   └── worldclim-derived              │   │   └── [ 0.359G]
│   ├── meso-scale                         │   ├── [31G]
│   │   ├── forest-height                  │   │   ├── [ 2.2G]
│   │   ├── glclu2019                      │   │   ├── [ 2.2G]
│   │   ├── glhymps                        │   │   ├── [ 0.054G]
│   │   ├── hydrolakes                     │   │   ├── [ 0.014G]
│   │   ├── lai                            │   │   ├── [17G]
│   │   ├── lgrip30                        │   │   ├── [ 0.954G]
│   │   ├── merit                          │   │   ├── [ 1.7G]
│   │   ├── modis-land                     │   │   ├── [ 0.380G]
│   │   ├── pelletier                      │   │   ├── [ 0.036G]
│   │   ├── soilgrids                      │   │   ├── [ 7.2G]
│   │   └── worldclim-derived              │   │   ├── [ 0.122G]
│   └── _metadata                          │   └── [0.0035G]
├── observations                           ├── [8.0G]
│   ├── headwater                          │   ├── [1.9G]
│   │   ├── obs-daily                      │   │   ├── [0.191G]
│   │   └── obs-hourly                     │   │   └── [1.7G]
│   ├── macro-scale                        │   ├── [1.7G]
│   │   ├── obs-daily                      │   │   ├── [0.28G]
│   │   └── obs-hourly                     │   │   └── [1.5G]
│   └── meso-scale                         │   ├── [4.5G]
│       ├── obs-daily                      │   │   ├── [0.5G]
│       └── obs-hourly                     │   │   └── [4G]
└── shapefiles                             └── [2.8G]
    ├── headwater                              ├── [0.21G]
    │   ├── delineation-outcomes               │   ├── [0.113G]
    │   ├── shapes-distributed                 │   ├── [0.011G]
    │   ├── shapes-forcing                     │   ├── [0.065G]
    │   ├── shapes-lumped                      │   ├── [0.0082G]
    │   └── shapes-reference                   │   └── [0.013G]
    ├── macro-scale                            ├── [1.7G]
    │   ├── delineation-outcomes               │   ├── [0.235G]
    │   ├── shapes-distributed                 │   ├── [0.264G]
    │   ├── shapes-forcing                     │   ├── [1.1G]
    │   ├── shapes-lumped                      │   ├── [0.034G]
    │   └── shapes-reference                   │   └── [0.077G]
    └── meso-scale                             └── [0.898G]
        ├── delineation-outcomes                    ├── [0.320G]
        ├── shapes-distributed                      ├── [0.087G]
        ├── shapes-forcing                          ├── [0.404G]
        ├── shapes-lumped                           ├── [0.027G]
        └── shapes-reference                        └── [0.061G]
```